<a href="https://colab.research.google.com/github/jwalx/Generative-AI/blob/main/Fine_Tuning_using_PEFT(parameter_effiicent_training).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
!pip install torch
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install loralib
!pip install peft

ERROR: Invalid requirement: '!pip'


In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer, GenerationConfig, TrainingArguments,Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [ ]:
model_name="google/flan-t5-base"

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name,torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
      all_model_params += param.numel()
      if param.requires_grad:
        trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\n all model parameters:{all_model_params}\n percentage of trainable mdoel parameters:{(trainable_model_params/all_model_params)*100} %\n"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
 all model parameters:247577856
 percentage of trainable mdoel parameters:100.0 %



In [ ]:
index =200

dialogue = dataset["test"][index]["dialogue"]
summary = dataset["test"][index]["summary"]

prompt = f"""
Summarize the following conversation>

{dialogue}

Summary:
"""
# adding prompt instead of dialogue
inputs = tokenizer(prompt , return_tensors = "pt")
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_token = True
)
dash_line = "-".join('' for x in range(100))

print(dash_line)
print(f"Input Prompt:\n{dialogue}")
print(dash_line)
print(f"Baseline human Summary:\n{summary}")
print(dash_line)
print(f"Model Generation - ZERO SHOT:\n{output}\n")

---------------------------------------------------------------------------------------------------
Input Prompt:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
---------------------------------------------------------------------------------------------------
Baseline human Sum

In [ ]:
def tokenize_function(example):
  start_prompt = "Summarize the following conversation.\n\n"
  end_prompt = "\n\nSummary:"
  prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
  example["input_ids"] = tokenizer(prompt,padding="max_length", truncation = True, return_tensors = "pt").input_ids
  example["labels"] = tokenizer(example["summary"],padding = "max_length",truncation = True, return_tensors = "pt").input_ids

  return example

tokenized_datasets = dataset.map(tokenize_function,batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["id","topic","dialogue","summary",])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example,index:index % 100==0,with_indices = True)

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Check the shapes of the datasets

print(f"shape of the datasets")
print(f"Training:{tokenized_datasets['train'].shape}")
print(f"Validation:{tokenized_datasets['validation'].shape}")
print(f"Test:{tokenized_datasets['test'].shape}")

print(tokenized_datasets)

shape of the datasets
Training:(125, 2)
Validation:(5, 2)
Test:(15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


# Performing Parameter Efficient Fine-Tuning(PEFT)

In [ ]:
# Setting up PEFT/LORA model for Fine-Tuning
from peft import LoraConfig,get_peft_model,TaskType

lora_config = LoraConfig(
    r= 32, #Rank
    lora_alpha = 32,
    target_modules = ["q","v"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = TaskType.SEQ_2_SEQ_LM  # FLAN-T5
)

In [ ]:
peft_model  =get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
 all model parameters:251116800
 percentage of trainable mdoel parameters:1.4092820552029972 %



# Train PEFT Adapter

In [ ]:
output_dir = f"/content/sample_data/peft-dialogue-summary-training-{str(int(time.time()))}"

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate = 1e-3,
    num_train_epochs = 1,
    logging_steps = 1,
    max_steps = 1
)

peft_trainer = Trainer(
    model = peft_model,
    args = peft_training_args,
    train_dataset = tokenized_datasets["train"],
)

In [ ]:
peft_trainer.train()

peft_model_path = "/content/sample_data/peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
index =200

dialogue = dataset["test"][index]["dialogue"]
summary = dataset["test"][index]["summary"]

prompt = f"""
Summarize the following conversation>

{dialogue}

Summary:
"""
input_ids = tokenizer(prompt , return_tensors = "pt").input_ids

peft_model_outputs =peft_model.generate(
        input_ids=input_ids,
        generation_config = GenerationConfig(max_new_tokens=200,num_beans=1))
peft_output = tokenizer.decode(peft_model_outputs[0],
        skip_special_token = True)

dash_line = "-".join('' for x in range(100))

print(dash_line)
print(f"Input Prompt:\n{dialogue}")
print(dash_line)
print(f"Baseline human Summary:\n{summary}")
print(dash_line)
print(f"PEFT-Model Generation - ZERO SHOT:\n{peft_output}\n")